In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Drop duplicate elements
data = pd.read_csv('../Data/data_original.csv').copy()
data = data.drop(columns=['Mặt tiền', 'Hướng nhà', 'Hướng ban công', 'Thông tin khác'])
data = data.drop_duplicates()
data = data[data['Tỉnh/Thành phố'].str.contains('Hà Nội', case=False, na=False)].reset_index(drop=True)
data = data[~data['Mức giá'].str.contains('Thỏa thuận', case=False, na=False)].reset_index(drop=True)
data = data[~data['Số phòng ngủ'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Số toilet'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Pháp lý'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data = data[~data['Nội thất'].str.contains('Không có thông tin', case=False, na=False)].reset_index(drop=True)
data

In [ ]:
# Clean area Data
area = data['Diện tích']
area = area.str.replace(' m²', '').str.replace('.', '').str.replace(',', '.').astype(np.float32)

In [ ]:
# Clean price Data

price = data['Mức giá']
# Convert to price per square
price_area = price.str.contains('tỷ', case=False, na=False)
cleaned_price_area = data.loc[price_area, 'Mức giá'].str.replace(' tỷ', '').str.replace(',', '.').astype(np.float32)*1000
price_per_square = (cleaned_price_area / area[price_area])
data.loc[price_area, 'Mức giá'] = price_per_square

# Clean form
wrong_form = price.str.contains(' triệu/m²', case=False, na=False)
cleaned_wrong_form = data.loc[wrong_form, 'Mức giá'].str.replace(' triệu/m²', '').str.replace(',', '.').astype(np.float32)
data.loc[wrong_form, 'Mức giá'] = cleaned_wrong_form

price

In [ ]:
# Clean room

bed_room = data['Số phòng ngủ']
bed_room = bed_room.str.replace(' phòng', '').astype(np.float32)

toilet_room = data['Số toilet']
toilet_room = toilet_room.str.replace(' phòng', '').astype(np.float32)


In [ ]:
# Clean legal

legal = data['Pháp lý']
wrong_form = legal.str.contains('HĐMB', case=False, na=False)
data.loc[wrong_form, 'Pháp lý'] = data.loc[wrong_form, 'Pháp lý'].str.replace('HĐMB', "Hợp đồng mua bán")



In [ ]:
cleaned_data = pd.DataFrame({
    'Xã/Phường': data['Xã/Phường'],
    'Quận/Huyện': data['Quận/Huyện'],
    'Chủ đầu tư': data['Chủ đầu tư'],
    'Tên dự án': data['Tên dự án'],
    'Diện tích (m²)': area,
    'Mức giá (triệu/m²)': price,
    'Số phòng ngủ (phòng)': bed_room,
    'Số toilet (phòng)': toilet_room,
    'Pháp lý': legal,
    'Nội thất': data['Nội thất'],
    'Lịch sử giá': data['Lịch sử giá'],
    'Khoảng giá': data['Khoảng giá']
})

cleaned_data.to_csv('../Data/cleaned_data.csv', index=False)

cleaned_data